In [1]:
import os 
import pandas as pd 
import time 
import datetime 
import logging
import json 
import random
import math

import warnings
warnings.filterwarnings("ignore")

from tqdm.autonotebook import tqdm
from collections import Counter
import gensim
import numpy as np 
import tensorflow as tf 

In [2]:
from utils import *

# 设置一些基础配置参数

In [3]:
class Config(dict):
    def __init__(self, path=None):
        super().__init__()
        ## 定义训练参数
        self['num_epochs'] = 6 
        self['evaluateEvery'] = 100 
        self['checkpointEvery'] = 100 
        self['learningRate'] = 0.001 
        
        ## 定义模型参数
        self['embeddingSize'] = 200 

        self['hiddenSizes'] = [256, 256]  # 单层LSTM的神经元数
        self['dropoutProb'] = 0.5 
        self['l2RegLambda'] = 0.0 
        
        ## 定义基础参数
        self['sequenceLength'] = 200 
        self['batch_size'] = 64 
        self['dataSource'] = path
        self['stopWordSource'] = "../data/english"
        self['numClasses'] = 1  
        self['train_size'] = 0.8   # 训练集和测试集比例
        self.threshold = 0.5 
        
        ## 保存模型参数
        self['checkpoint_dir'] = "../model/BiLSTM/imdb/checkpoint"
        self['summary_dir'] = "../model/BiLSTM/imdb/summary"
        self['max_to_keep'] = 5

BiLSTM的数据处理方式和TextCNN相同，都是以Word为单位的，所以这里直接使用之前写好的DataSet类即可

# 定义模型

## 定义BiLSTM模型类

In [4]:
class BiLSTM(BaseModel):
    def __init__(self, config, wordEmbedding):
        super(BiLSTM, self).__init__(config)
        self.wordEmbedding = wordEmbedding
        self.build_model()
        self.init_saver()
        
    def build_model(self):
        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, self.config["sequenceLength"]], name="inputX")
        self.inputY = tf.placeholder(tf.int32, [None], name="inputY")
        
        self.dropoutProb = tf.placeholder(tf.float32, name="dropoutProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):
            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(self.wordEmbedding, dtype=tf.float32, name="word2vec"),
                                name="W")
            # 利用此前向矩阵将输入的数据中的词替换为词向量
            # 维度为 [batch_size, sequence_length, embedding_size]
            self.embededWords = tf.nn.embedding_lookup(self.W, self.inputX)
        
        # 定义两层双向LSTM
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(self.config["hiddenSizes"]):
                with tf.name_scope(f"BiLSTM_{idx}"):
                    ## 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutProb)
                    ## 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutProb)
                    
                    ## 采用动态RNN，可以动态输入序列的长度，没有输入则取序列的全长
                    ## outputs是一个元组(output_fw, output_bw)
                    ## fw和bw的维度都是[batch, max_time, hidden_size]
                    outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(
                        lstmFwCell, lstmBwCell, self.embededWords, dtype=tf.float32,
                        scope=f"bilstm_{idx}")
                    ## 对outputs中的fw和bw结果拼接成 [batch, max_time, hidden_size*2]
                    self.embededWords = tf.concat(outputs, 2)
        
        # 分类只需要取出最后一个时间步的输出即可
        finalOutput = self.embededWords[:, -1, :]
        outputSize = self.config["hiddenSizes"][-1] * 2 ## 双向LSTM最后输出是fw和bw的拼接
        output = tf.reshape(finalOutput, [-1, outputSize])
        
        # 全连接层输出
        with tf.name_scope("output"):
            self.logits = tf.layers.dense(output, self.config['numClasses'], name="dense",
                                         kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                         bias_initializer=tf.constant_initializer(0.1))
            ## 获取全连接层的权重
            with tf.variable_scope("dense", reuse=True):
                outputW = tf.get_variable("kernel")
            l2Loss += tf.nn.l2_loss(outputW)
            
            if self.config['numClasses'] == 1: 
                self.predictions = tf.sigmoid(self.logits)
            elif self.config['numClasses'] > 1: 
                self.predictions = tf.nn.softmax(self.logits, dim=1)
            
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            if self.config["numClasses"] == 1: 
                losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits,
                                                                labels=tf.cast(tf.reshape(self.inputY, [-1, 1]),
                                                                              dtype=tf.float32))
            elif self.config['numClasses'] > 1: 
                losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits,
                                                                       labels=self.inputY)
            
            self.loss = tf.reduce_mean(losses) + self.config["l2RegLambda"] * l2Loss
            
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                self.train_op = tf.train.AdamOptimizer(
                    self.config['learningRate']).minimize(self.loss, global_step=self.global_step_tensor)
    
    def init_saver(self):
        '''
        初始化用于保存模型的对象
        '''
        self.saver = tf.train.Saver(max_to_keep=self.config['max_to_keep'])

## 定义训练类

In [5]:
class Trainer(BaseTrain):
    def __init__(self, sess, model, data, config, logger):
        '''
        这里的data要求是元组的形式，data[0]是train对象，data[1]是eval对象
        '''
        super(Trainer, self).__init__(sess, model, data, config, logger)
        self.train = data[0]
        self.eval = data[1]
        
    def train_epoch(self):
        # 定义迭代次数
        num_iter_per_epoch = self.train.length // self.config["batch_size"]
        
        for _ in tqdm(range(num_iter_per_epoch)):
            loss, metrics, step = self.train_step()
            train_acc, train_f_score = metrics['accuracy'], metrics['f_beta']
            
            ## 将训练过程的损失写入
            summaries_dict = {"loss": loss, 
                             "acc": np.array(train_acc), 
                             "f_score": np.array(train_f_score)}
            self.logger.summarize(step, summarizer="train", scope="train_summary",
                                      summaries_dict=summaries_dict)
            
            if step % self.config["evaluateEvery"] == 0: 
                print("Train —— Step: {} | Loss: {} | Acc: {} | F1_Score: {}".format(
                    step, loss, train_acc, train_f_score))
                # 对测试集进行评估
                eval_losses = []
                eval_true = []
                eval_pred = []
                for batchEval in self.eval.iter_all(self.config["batch_size"]):
                    loss, predictions = self.eval_step(batchEval[0], batchEval[1])
                    eval_losses.append(loss)
                    eval_true.extend(batchEval[1])
                    eval_pred.extend(predictions)
                    
                getMetric = Metric(np.array(eval_pred), np.array(eval_true), self.config)
                metrics = getMetric.get_metrics()
                loss_mean = np.round(np.mean(eval_losses), 5)
                prec_mean = np.round(metrics['precision'], 5)
                recall_mean = np.round(metrics['recall'], 5)
                
                time_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S %p")
                
                print("{} | Loss: {} | Precision: {} | Recall: {}".format(time_str,
                                                                         loss_mean,
                                                                         prec_mean,
                                                                         recall_mean))
                summaries_dict = {"loss": np.array(loss_mean),
                                 "precision": np.array(prec_mean),
                                 "recall": np.array(recall_mean)}
                self.logger.summarize(step, summarizer="test", scope="test_summary",
                                     summaries_dict=summaries_dict)
            if step % self.config["checkpointEvery"] == 0: 
                self.model.save(self.sess)
        
    
    def train_step(self):
        batch_x, batch_y = next(self.train.next_batch(self.config["batch_size"]))
        feed_dict = {self.model.inputX: batch_x, self.model.inputY: batch_y,
                    self.model.dropoutProb: self.config["dropoutProb"]}
        _, loss, predictions, step = self.sess.run([self.model.train_op,
                                                   self.model.loss,
                                                   self.model.predictions,
                                                   self.model.global_step_tensor],
                                                  feed_dict=feed_dict)
        getMetric = Metric(predictions, batch_y, self.config)
        metrics = getMetric.get_metrics()
        
        return loss, metrics, step 
    
    def eval_step(self, batch_x, batch_y):
        '''
        使用验证集测试
        '''
        feed_dict = {self.model.inputX: batch_x, self.model.inputY: batch_y,
                    self.model.dropoutProb: 1}
        loss, predictions = self.sess.run([self.model.loss, self.model.predictions],
                                         feed_dict=feed_dict)
        
            
        return loss, predictions

# 使用数据集进行训练

## 使用IMDB数据集

In [6]:
def main():
    path = "../data/imdb/labeldTrain.csv"
    config = Config(path)
    
    create_dirs([config["summary_dir"], config["checkpoint_dir"]])
    
    data = Dataset(config)
    
    ## 生成训练集数据
    data.dataGen("../data/imdb/", prefix="imdb")
    
    train_X, train_y, eval_X, eval_y = data.trainReviews, data.trainLabels, data.evalReviews, data.evalLabels
    wordEmbedding, labels = data.wordEmbedding, data.labelList
    
    train_data = DataGenerator(train_X, train_y)
    eval_data = DataGenerator(eval_X, eval_y)
    pack_data = [train_data, eval_data]
    
    tf.reset_default_graph()
    
    # 设置计算图的配置
    session_conf = tf.ConfigProto(allow_soft_placement=True, 
                                 log_device_placement=False)
    session_conf.gpu_options.allow_growth = True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9 
    sess = tf.Session(config=session_conf)
    
    ## 创建一个实例
    model = BiLSTM(config, wordEmbedding)
    
    logger = Logger(sess, config)
    trainer = Trainer(sess, model, pack_data, config, logger)
    
    trainer.train_all()

In [7]:
main()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.

当前正处于第1次迭代


Train —— Step: 100 | Loss: 0.6788748502731323 | Acc: 0.53125 | F1_Score: 0.64286
2019-08-07 16:13:23 PM | Loss: 0.6908699870109558 | Precision: 0.50661 | Recall: 0.99963
Saving model...
Model saved
Train —— Step: 200 | Loss: 0.7004097700119019 | Acc: 0.51562 | F1_Score: 0.4151
2019-08-07 16:14:09 PM | Loss: 0.6825199723243713 | Precision: 0.65393 | Recall: 0.1799
Saving model...
Model saved
Train —— Step: 300 | Loss: 0.653802752494812 | Acc: 0.625 | F1_Score: 0.5
2019-08-07 16:14:56 PM | Loss: 0.6627399921417236 | Precision: 0.6888 | Recall: 0.28096
Saving model...
Model saved


当前正处于第2次迭代


Train —— Step: 400 | Loss: 0.6164913177490234 | Acc: 0.67188 | F1_Score: 0.75862
2019-08-07 16:15:43 PM | Loss: 0.6803799867630005 | Precision: 0.59018 | Recall: 0.96562
Saving model...
Model saved
Train —— Step: 500 | Loss: 0.6778290271759033 | Acc: 0.57812 | F1_Score: 0.69663
2019-08-07 16:16:30 PM | Loss: 0.690280020236969 | Precision: 0.51094 | Recall: 0.8462
Saving model...
Model saved
Train —— Step: 600 | Loss: 0.6390670537948608 | Acc: 0.60938 | F1_Score: 0.62686
2019-08-07 16:17:16 PM | Loss: 0.6619700193405151 | Precision: 0.81848 | Recall: 0.38994
Saving model...
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Model saved


当前正处于第3次迭代


Train —— Step: 700 | Loss: 0.5293903350830078 | Acc: 0.6875 | F1_Score: 0.81132
2019-08-07 16:18:00 PM | Loss: 0.6859999895095825 | Precision: 0.52036 | Recall: 0.99347
Saving model...
Model saved
Train —— Step: 800 | Loss: 0.32116228342056274 | Acc: 0.84375 | F1_Score: 0.83871
2019-08-07 16:18:45 PM | Loss: 0.3912000060081482 | Precision: 0.8764 | Recall: 0.78488
Saving model...
Model saved
Train —— Step: 900 | Loss: 0.2669948637485504 | Acc: 0.90625 | F1_Score: 0.91176
2019-08-07 16:19:30 PM | Loss: 0.33910998702049255 | Precision: 0.83958 | Recall: 0.89099
Saving model...
Model saved


当前正处于第4次迭代


Train —— Step: 1000 | Loss: 0.2796424329280853 | Acc: 0.875 | F1_Score: 0.88889
2019-08-07 16:20:17 PM | Loss: 0.3266099989414215 | Precision: 0.89983 | Recall: 0.82441
Saving model...
Model saved
Train —— Step: 1100 | Loss: 0.11097978055477142 | Acc: 0.95312 | F1_Score: 0.95082
2019-08-07 16:21:04 PM | Loss: 0.3210200071334839 | Precision: 0.90079 | Recall: 0.83189
Saving model...
Model saved
Train —— Step: 1200 | Loss: 0.10077868402004242 | Acc: 0.96875 | F1_Score: 0.96667
2019-08-07 16:21:50 PM | Loss: 0.36357998847961426 | Precision: 0.9098 | Recall: 0.81362
Saving model...
Model saved


当前正处于第5次迭代


Train —— Step: 1300 | Loss: 0.05362792685627937 | Acc: 1.0 | F1_Score: 1.0
2019-08-07 16:22:37 PM | Loss: 0.35071998834609985 | Precision: 0.87359 | Recall: 0.87824
Saving model...
Model saved
Train —— Step: 1400 | Loss: 0.2135436236858368 | Acc: 0.95312 | F1_Score: 0.95238
2019-08-07 16:23:26 PM | Loss: 0.42054998874664307 | Precision: 0.92283 | Recall: 0.78202
Saving model...
Model saved
Train —— Step: 1500 | Loss: 0.08707692474126816 | Acc: 0.96875 | F1_Score: 0.96428
2019-08-07 16:24:15 PM | Loss: 0.3658500015735626 | Precision: 0.86347 | Recall: 0.90634
Saving model...
Model saved


当前正处于第6次迭代


Train —— Step: 1600 | Loss: 0.08929283916950226 | Acc: 0.96875 | F1_Score: 0.97369
2019-08-07 16:25:04 PM | Loss: 0.35175999999046326 | Precision: 0.93157 | Recall: 0.74715
Saving model...
Model saved
Train —— Step: 1700 | Loss: 0.170895516872406 | Acc: 0.95312 | F1_Score: 0.94915
2019-08-07 16:25:52 PM | Loss: 0.5068100094795227 | Precision: 0.93019 | Recall: 0.76063
Saving model...
Model saved
Train —— Step: 1800 | Loss: 0.041507914662361145 | Acc: 0.98438 | F1_Score: 0.98462
2019-08-07 16:26:40 PM | Loss: 0.3705100119113922 | Precision: 0.88491 | Recall: 0.87154
Saving model...
Model saved



- 最终结果 —— Precision: 0.8849, Recall: 0.87154

## 使用yelps数据集

In [9]:
def main():
    path = "../data/yelps/yelps_test.csv"
    config = Config(path)
    config["summary_dir"] = "../model/BiLSTM/yelps/summary"
    config["checkpoint_dir"] = "../model/BiLSTM/yelps/checkpoint"
    config['evaluateEvery'] = 500 
    config['checkpointEvery'] = 500 
    
    
    create_dirs([config["summary_dir"], config["checkpoint_dir"]])
    
    data = Dataset(config)
    
    ## 生成训练集数据
    data.dataGen("../data/yelps/", prefix="yelps")
    
    train_X, train_y, eval_X, eval_y = data.trainReviews, data.trainLabels, data.evalReviews, data.evalLabels
    wordEmbedding, labels = data.wordEmbedding, data.labelList
    
    train_data = DataGenerator(train_X, train_y)
    eval_data = DataGenerator(eval_X, eval_y)
    pack_data = [train_data, eval_data]
    
    tf.reset_default_graph()
    
    # 设置计算图的配置
    session_conf = tf.ConfigProto(allow_soft_placement=True, 
                                 log_device_placement=False)
    session_conf.gpu_options.allow_growth = True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9 
    sess = tf.Session(config=session_conf)
    
    ## 创建一个实例
    model = BiLSTM(config, wordEmbedding)
    
    logger = Logger(sess, config)
    trainer = Trainer(sess, model, pack_data, config, logger)
    
    trainer.train_all()

In [10]:
main()


当前正处于第1次迭代


Train —— Step: 500 | Loss: 0.602176308631897 | Acc: 0.71875 | F1_Score: 0.83334
2019-08-07 16:40:32 PM | Loss: 0.6355599761009216 | Precision: 0.67384 | Recall: 0.95684
Saving model...
Model saved
Train —— Step: 1000 | Loss: 0.5355428457260132 | Acc: 0.75 | F1_Score: 0.79487
2019-08-07 16:48:48 PM | Loss: 0.592739999294281 | Precision: 0.81538 | Recall: 0.65269
Saving model...
Model saved
Train —— Step: 1500 | Loss: 0.6322532892227173 | Acc: 0.67188 | F1_Score: 0.79208
2019-08-07 16:57:25 PM | Loss: 0.6322199702262878 | Precision: 0.6709 | Recall: 0.98431
Saving model...
Model saved
Train —— Step: 2000 | Loss: 0.265129417181015 | Acc: 0.90625 | F1_Score: 0.93333
2019-08-07 17:05:29 PM | Loss: 0.344539999961853 | Precision: 0.89429 | Recall: 0.91869
Saving model...
Model saved
Train —— Step: 2500 | Loss: 0.3048836886882782 | Acc: 0.90625 | F1_Score: 0.9375
2019-08-07 17:13:41 PM | Loss: 0.2709900140762329 | Precision: 0.91541 | Recall: 0.92105
Saving model...
Model saved
Train —— Step: 

Train —— Step: 9000 | Loss: 0.3147425651550293 | Acc: 0.89062 | F1_Score: 0.90666
2019-08-07 19:01:06 PM | Loss: 0.22346000373363495 | Precision: 0.92731 | Recall: 0.93644
Saving model...
Model saved
Train —— Step: 9500 | Loss: 0.21113228797912598 | Acc: 0.90625 | F1_Score: 0.93023
2019-08-07 19:09:16 PM | Loss: 0.23260000348091125 | Precision: 0.90981 | Recall: 0.95605
Saving model...
Model saved
Train —— Step: 10000 | Loss: 0.1315956711769104 | Acc: 0.95312 | F1_Score: 0.96296
2019-08-07 19:17:26 PM | Loss: 0.22418999671936035 | Precision: 0.93324 | Recall: 0.92839
Saving model...
Model saved
Train —— Step: 10500 | Loss: 0.22049115598201752 | Acc: 0.875 | F1_Score: 0.9
2019-08-07 19:25:39 PM | Loss: 0.22544999420642853 | Precision: 0.904 | Recall: 0.96269
Saving model...
Model saved
Train —— Step: 11000 | Loss: 0.3426017463207245 | Acc: 0.85938 | F1_Score: 0.88889
2019-08-07 19:33:48 PM | Loss: 0.22436000406742096 | Precision: 0.91202 | Recall: 0.95595
Saving model...
Model saved
Tra

Train —— Step: 18000 | Loss: 0.20320135354995728 | Acc: 0.90625 | F1_Score: 0.925
2019-08-07 21:28:30 PM | Loss: 0.22353999316692352 | Precision: 0.91757 | Recall: 0.95004
Saving model...
Model saved
Train —— Step: 18500 | Loss: 0.3349512219429016 | Acc: 0.875 | F1_Score: 0.90476
2019-08-07 21:36:36 PM | Loss: 0.22429999709129333 | Precision: 0.92588 | Recall: 0.93966
Saving model...
Model saved
Train —— Step: 19000 | Loss: 0.1569259762763977 | Acc: 0.9375 | F1_Score: 0.95122
2019-08-07 21:44:43 PM | Loss: 0.22247999906539917 | Precision: 0.91913 | Recall: 0.94833
Saving model...
Model saved
Train —— Step: 19500 | Loss: 0.18530502915382385 | Acc: 0.90625 | F1_Score: 0.9375
2019-08-07 21:52:56 PM | Loss: 0.22425000369548798 | Precision: 0.92476 | Recall: 0.94103
Saving model...
Model saved
Train —— Step: 20000 | Loss: 0.3287199139595032 | Acc: 0.85938 | F1_Score: 0.87672
2019-08-07 22:01:04 PM | Loss: 0.22673000395298004 | Precision: 0.92381 | Recall: 0.94239
Saving model...
Model saved

Train —— Step: 27000 | Loss: 0.08539923280477524 | Acc: 0.96875 | F1_Score: 0.98039
2019-08-07 23:56:48 PM | Loss: 0.23725999891757965 | Precision: 0.91834 | Recall: 0.94773
Saving model...
Model saved
Train —— Step: 27500 | Loss: 0.16274616122245789 | Acc: 0.9375 | F1_Score: 0.94872
2019-08-08 00:05:37 AM | Loss: 0.2337000072002411 | Precision: 0.92583 | Recall: 0.93624
Saving model...
Model saved
Train —— Step: 28000 | Loss: 0.1835632026195526 | Acc: 0.95312 | F1_Score: 0.96629
2019-08-08 00:14:07 AM | Loss: 0.23868000507354736 | Precision: 0.93032 | Recall: 0.92814
Saving model...
Model saved
Train —— Step: 28500 | Loss: 0.15302756428718567 | Acc: 0.95312 | F1_Score: 0.96703
2019-08-08 00:22:40 AM | Loss: 0.2508000135421753 | Precision: 0.92451 | Recall: 0.93367
Saving model...
Model saved
Train —— Step: 29000 | Loss: 0.10080263763666153 | Acc: 0.96875 | F1_Score: 0.97778
2019-08-08 00:30:58 AM | Loss: 0.25014999508857727 | Precision: 0.92737 | Recall: 0.93385
Saving model...
Model 

Train —— Step: 36000 | Loss: 0.03554639592766762 | Acc: 0.98438 | F1_Score: 0.98795
2019-08-08 02:32:54 AM | Loss: 0.2669999897480011 | Precision: 0.91977 | Recall: 0.943
Saving model...
Model saved
Train —— Step: 36500 | Loss: 0.11708894371986389 | Acc: 0.95312 | F1_Score: 0.96774
2019-08-08 02:41:33 AM | Loss: 0.26875999569892883 | Precision: 0.92794 | Recall: 0.92824
Saving model...
Model saved
Train —— Step: 37000 | Loss: 0.0441308319568634 | Acc: 0.98438 | F1_Score: 0.98666
2019-08-08 02:50:13 AM | Loss: 0.26993998885154724 | Precision: 0.92365 | Recall: 0.93188
Saving model...
Model saved
Train —— Step: 37500 | Loss: 0.0383145734667778 | Acc: 0.98438 | F1_Score: 0.98823
2019-08-08 02:58:48 AM | Loss: 0.2779400050640106 | Precision: 0.9383 | Recall: 0.90425
Saving model...
Model saved
Train —— Step: 38000 | Loss: 0.11477693170309067 | Acc: 0.96875 | F1_Score: 0.97297
2019-08-08 03:06:46 AM | Loss: 0.26673999428749084 | Precision: 0.92168 | Recall: 0.93544
Saving model...
Model sav

Train —— Step: 45000 | Loss: 0.035265251994132996 | Acc: 1.0 | F1_Score: 1.0
2019-08-08 05:08:46 AM | Loss: 0.27807000279426575 | Precision: 0.92854 | Recall: 0.92219
Saving model...
Model saved
Train —— Step: 45500 | Loss: 0.12857815623283386 | Acc: 0.96875 | F1_Score: 0.97619
2019-08-08 05:17:06 AM | Loss: 0.314300000667572 | Precision: 0.91819 | Recall: 0.93423
Saving model...
Model saved
Train —— Step: 46000 | Loss: 0.12163420766592026 | Acc: 0.92188 | F1_Score: 0.94117
2019-08-08 05:25:56 AM | Loss: 0.2884100079536438 | Precision: 0.92187 | Recall: 0.93216
Saving model...
Model saved
Train —— Step: 46500 | Loss: 0.058673374354839325 | Acc: 0.96875 | F1_Score: 0.97436
2019-08-08 05:34:28 AM | Loss: 0.29844000935554504 | Precision: 0.92269 | Recall: 0.93001
Saving model...
Model saved
Train —— Step: 47000 | Loss: 0.02879524976015091 | Acc: 1.0 | F1_Score: 1.0
2019-08-08 05:44:10 AM | Loss: 0.32006001472473145 | Precision: 0.92741 | Recall: 0.92079
Saving model...
Model saved
Train —

最终结果 —— Precision: 0.91988, Recall: 0.92888